In [1]:
#import modules
import psycopg2 
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pickle 

In [2]:
#import and clean data from MovieLens dataset, prepare for use with imdb API
Movie_list = pd.read_csv('Movies.csv', header = 0)

def deleteLastspace(title):
    return title.rstrip()

def deleteThe(title):
    title = title.replace(', The ', '')
    if title == ', The ':
        return title.replace(', The ', '')
    else:
        return title
    
def deleteA(title):
    title = title.replace(', A ', '')
    if title == ', A':
        return title.replace(', A ', '')
    else:
        return title
    
def deleteAn(title):
    title = title.replace(', An ', '')
    if title == ', An ':
        return title.replace(', An ', '')
    else:
        return title
    
def deleteLe(title):
    title = title.replace(', Le ', '')
    if title == ', Le ':
        return title.replace(', Le ', '')
    else:
        return title
    
def deleteLa(title):
    title = title.replace(', La ', '')
    if title == ', La ':
        return title.replace(', La ', '')
    else:
        return title

def deleteEl(title):
    title = title.replace(', El ', '')
    if title == ', El ':
        return title.replace(', El ', '')
    else:
        return title

def deleteLes(title):
    title = title.replace(', Les', '')
    if title == ', Les':
        return title.replace(', Les', '')
    else:
        return title

def deleteLastplus(title):
    if title[-1] == '+':
        return title[0:-1]
    else:
        return title

#extracts year from end of title in parentheses and creates new column with year
Movie_list['year'] = Movie_list['title'].str.extract('(\d\d\d\d)', expand=True)

#title that excludes the year
Movie_list['title'] = Movie_list['title'].astype(str).str[:-6]

#removes everything in parentheses
Movie_list['title'] = Movie_list['title'].str.replace(r"\(.*\)","")

#deletes the ', The' at end of title (consider adding other cases into this function)
Movie_list['title'] = Movie_list['title'].apply(deleteThe).apply(deleteLes)\
                    .apply(deleteLe).apply(deleteA).apply(deleteAn).apply(deleteLa)\
                    .apply(deleteEl)
Movie_list['title'] = Movie_list['title'].apply(deleteLastspace)
Movie_list['title'] = Movie_list['title'].str.replace(" ","+")
Movie_list['api'] = Movie_list['title'] + '&y='
Movie_list['api-year'] = Movie_list['title'] + '&y=' + Movie_list['year']

In [3]:
#ratings info/pickle for later/building model
Data_for_model = pd.read_csv('ratings.csv', header = 0)
with open('Data_for_model.pkl', 'w') as picklefile:
    pickle.dump(Data_for_model, picklefile)

In [4]:
#resources

#http://virendra.me/scraping-imdb-with-python/
#https://github.com/richardasaurus/imdb-pie
#https://github.com/jameskang410/scraping-netflix
#https://www.cinesift.com/#/
#http://www.hbo.com/movies/catalog

In [5]:
#showtime/webscrape data-go back through later to clean up 'links' list
import requests
from bs4 import BeautifulSoup
from IPython.core.display import HTML
from selenium import webdriver
links = ['http://www.sho.com/movies/action', 'http://www.sho.com/movies/action/page/2', \
         'http://www.sho.com/movies/action/page/3', 'http://www.sho.com/movies/comedy', \
         'http://www.sho.com/movies/comedy/page/2', 'http://www.sho.com/movies/comedy/page/3', \
         'http://www.sho.com/movies/comedy/page/4', 'http://www.sho.com/movies/comedy/page/5', \
         'http://www.sho.com/movies/documentary', 'http://www.sho.com/movies/documentary/page/2', \
         'http://www.sho.com/movies/drama', 'http://www.sho.com/movies/drama/page/2', \
         'http://www.sho.com/movies/drama/page/3', 'http://www.sho.com/movies/drama/page/4', \
         'http://www.sho.com/movies/drama/page/5', 'http://www.sho.com/movies/drama/page/6', \
         'http://www.sho.com/movies/drama/page/7', 'http://www.sho.com/movies/family', \
         'http://www.sho.com/movies/horror', 'http://www.sho.com/movies/horror/page/2', \
         'http://www.sho.com/movies/music', 'http://www.sho.com/movies/sci-fi-fantasy', \
         'http://www.sho.com/movies/sci-fi-fantasy/page/2', 'http://www.sho.com/movies/suspense', \
         'http://www.sho.com/movies/suspense/page/2', 'http://www.sho.com/movies/suspense/page/3', \
         'http://www.sho.com/movies/suspense/page/4', 'http://www.sho.com/movies/suspense/page/5', \
         'http://www.sho.com/movies/suspense/page/6', 'http://www.sho.com/movies/suspense/page/7']

showtime = []

#loop through each link, scrape movie titles and append to 'showtime' list
for link in links:
    show_movie = requests.get(link)
    show_soup = BeautifulSoup(show_movie.content, "lxml")
    titles = show_soup.findAll(class_='movies-gallery__title')
    for title in titles:
        if title in showtime:
            continue
        else:
            showtime.append([title.text, 'showtime'])

#convert into dataframe, in service column populate with 'showtime'
showtime_df = pd.DataFrame(data = showtime, columns = ['title', 'service'])

In [6]:
#import openpyxl module that imports data from excel files
import openpyxl

netflix_movies = []

#open file, append value for every row in column two and 'netflix' which will be used in 
#service column in dataframe
netflix = openpyxl.load_workbook('NetflixMovies.xlsx')
sheet = netflix.get_sheet_by_name('Movies')
for x in range(1,4393):
    netflix_movies.append([sheet.cell(row=x, column=2).value, 'netflix'])

#convert list of lists into dataframe
netflix_df = pd.DataFrame(data = netflix_movies, columns = ['title', 'service'])

//anaconda/lib/python2.7/site-packages/openpyxl/reader/worksheet.py:322: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [7]:
#repeat process for hulu
hulu_movies = []

hulu = openpyxl.load_workbook('HuluMovies.xlsx')
sheet = hulu.get_sheet_by_name('Sheet1')
for x in range(2,1090):
    hulu_movies.append([sheet.cell(row=x, column=4).value, 'hulu'])

hulu_df = pd.DataFrame(data = hulu_movies, columns = ['title', 'service'])

In [8]:
#amazon
amazon_movies = []

amazon = openpyxl.load_workbook('AmazonMovies.xlsx')
sheet = amazon.get_sheet_by_name('Sheet1')
for x in range(2,2537):
    sheet.cell(row=x, column=1).value
    amazon_movies.append(sheet.cell(row=x, column=1).value)

#remove duplicates which have space in front of them
Amazon_Movies = []
for line in amazon_movies:
    if line[0] == " ":
        continue
    else:
        Amazon_Movies.append([line, 'amazon'])

amazon_df = pd.DataFrame(data = Amazon_Movies, columns = ['title', 'service'])

In [9]:
#hbo
HBO_movies = []

HBO = openpyxl.load_workbook('HBOMovies.xlsx')
sheet = HBO.get_sheet_by_name('Sheet1')
for x in range(2,345):
    sheet.cell(row=x, column=2).value
    HBO_movies.append([sheet.cell(row=x, column=2).value, 'HBO'])


HBO_df = pd.DataFrame(data = HBO_movies, columns = ['title', 'service'])

In [10]:
#rotten tomatoes, added for more recent list of movies
tomatoes_movies = []

#left second value in list empty because rotten tomatoes not streaming service
tomatoes = openpyxl.load_workbook('movie_metadata.xlsx')
sheet = tomatoes.get_sheet_by_name('sheet1')
for x in range(2,5045):
    tomatoes_movies.append([sheet.cell(row=x, column=12).value, ''])
    
for x in tomatoes_movies:
    for y in x:
        if y in ['c', 'd' ]:
            x = x.replace(y, "")
    
tomatoes_df = pd.DataFrame(data = tomatoes_movies, columns = ['title', 'service'])

In [11]:
#create list of all of the streaming services' dataframes + rotten tomatoes
services = [HBO_df, showtime_df, amazon_df, netflix_df, hulu_df, tomatoes_df]

#concatanate dataframes in large dataframe 'Movies_services'
Movies_services = pd.concat(services)

#clean dataframe, create 'year' column
Movies_services['year'] = Movies_services['title'].str.extract('(\d\d\d\d)', expand=True)
Movies_services['title'] = Movies_services['title'].str.replace(r"\(.*\)","")
Movies_services['title'] = Movies_services['title'].str.replace(" ","+")
length = len(Movies_services)
Movies_services['movieId'] = [x for x in range(140000, (140000 + length))]

Movies_services['api'] = Movies_services['title'] + '&y='
Movies_services['api-year'] = Movies_services['title'] + '&y='

#pickle Movies_services
import pickle
with open('Movies_services.pkl', 'w') as picklefile:
    pickle.dump(Movies_services, picklefile)


In [12]:
#open up pickled dataframe
with open('Movies_services.pkl', 'r') as picklefile:
    Movies_services = pickle.load(picklefile)

#outer merge Movie_list and Movie_services dataframes, final list of movies
Complete_df = pd.merge(Movie_list, Movies_services, how='outer')

#pickle complete dataframe
with open('complete_df.pkl', 'w') as picklefile:
    pickle.dump(Complete_df, picklefile)


//anaconda/lib/python2.7/site-packages/pandas/tools/merge.py:714: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  rlab = rizer.factorize(rk)


In [13]:
#list of column names from one dictionary of data
#testing api

attributes = []

from urllib2 import urlopen

movies = urlopen("http://www.omdbapi.com/?t=Catwalk&y=&plot=short&r=json&tomatoes=true")
response = movies.read()

#convert to dictionary
response_dict = eval(response)

#append each key in dictionary to a list, will be our model features
for key in response_dict.keys():
    attributes.append(key)

In [14]:
#extremely slow way to populate dataframe with info from api

#for attribute in attributes:
    #name = attribute
    #attribute = []
    #for item, row in abr_df.iterrows():
        #try:
            #api = row['api']
            #movies = urlopen("http://www.omdbapi.com/?t={}&plot=short&r=json&tomatoes=true".format(api))
            #results = movies.read()
            #response_dict = eval(results)
            #place_hold = response_dict[name]
            #attribute.append(place_hold)
        #except:
            #attribute.append(None)
            #pass
    #abr_df[name] = attribute
#abr_df


In [15]:
#populate final dataframe with api

with open('complete_df.pkl', 'r') as picklefile:
    Complete_df = pickle.load(picklefile)

#define function that will be applied to every row in the database
def pop_df(data):
    #initiate empty lists that will each be populated as we cycle through rows in dataframe
    Plot = []
    Rated = []
    Language = []
    Title = []
    Country = []
    tomatoMeter = []
    Writer = []
    tomatoURL = []
    BoxOffice = []
    tomatoUserRating = []
    Production = []
    Actors = []
    tomatoFresh = []
    Runtime = []
    Type = []
    imdbVotes = []
    Website = []
    tomatoConsensus = []
    tomatoReviews = []
    tomatoImage = []
    Poster = []
    tomatoRotten = []
    Metascore = []
    Response = []
    Director = []
    Released = []
    tomatoUserReviews = []
    Awards = []
    tomatoRating = []
    Year = []
    Genre = []
    tomatoUserMeter = []
    DVD = []
    imdbRating = []
    imdbID = []
    for item, row in data.iterrows():
        #iterate through rows in dataframe
        try:    
            #set api to value in 'api-year' column with was formated to be directly entered
            #into imdb api
            api = row['api-year']   #pass 'api' variable into url
            movies = urlopen("http://www.omdbapi.com/?t={}&plot=short&r=json&tomatoes=true".format(api))
            results = movies.read()   #create dictionary of the results
            response_dict = eval(results)
            response = []   #create empty list
            for key in response_dict.keys():   #append keys to response dictionary
                response.append(key)
            if set(response) != set(attributes):   #make sure movie has all features
                try:   #if it doesn't try using api w/o year (sometimes year incorrect and throws off api)
                    api = row['api']
                    movies = urlopen("http://www.omdbapi.com/?t={}&plot=short&r=json&tomatoes=true".format(api))
                    results = movies.read()
                    response_dict = eval(results)
                    response = []
                    for key in response_dict.keys():
                        response.append(key)
                    if set(response) >= set(attributes):   #if api works w/o year, append value in dictionary
                                                           #whose key corresponds to list
                        rated = response_dict['Rated']
                        Rated.append(rated)
                        plot = response_dict['Plot']
                        Plot.append(plot)
                        language = response_dict['Language']
                        Language.append(language)
                        title = response_dict['Title']
                        Title.append(title)
                        country = response_dict['Country']
                        Country.append(country)
                        tm = response_dict['tomatoMeter']
                        tomatoMeter.append(tm)
                        wr = response_dict['Writer']
                        Writer.append(wr)
                        turl = response_dict['tomatoURL']
                        tomatoURL.append(turl)
                        BO = response_dict['BoxOffice']
                        BoxOffice.append(BO)
                        tUR = response_dict['tomatoUserRating']
                        tomatoUserRating.append(tUR)
                        pro = response_dict['Production']
                        Production.append(pro)
                        act = response_dict['Actors']
                        Actors.append(act)
                        tf = response_dict['tomatoFresh']
                        tomatoFresh.append(tf)
                        rt = response_dict['Runtime']
                        Runtime.append(rt)
                        ty = response_dict['Type']
                        Type.append(ty)
                        imdbV = response_dict['imdbVotes']
                        imdbVotes.append(imdbV)
                        tomC = response_dict['tomatoConsensus']
                        tomatoConsensus.append(tomC)
                        tomR = response_dict['tomatoReviews']
                        tomatoReviews.append(tomR)
                        tomI = response_dict['tomatoImage']
                        tomatoImage.append(tomI)
                        post = response_dict['Poster']
                        Poster.append(post)
                        tomRot = response_dict['tomatoRotten']
                        tomatoRotten.append(tomRot)
                        meta = response_dict['Metascore']
                        Metascore.append(meta)
                        resp = response_dict['Response']
                        Response.append(resp)
                        dire = response_dict['Director']
                        Director.append(dire)
                        rele = response_dict['Released']
                        Released.append(rele)
                        tomURev = response_dict['tomatoUserReviews']
                        tomatoUserReviews.append(tomURev)
                        aw = response_dict['Awards']
                        Awards.append(aw)
                        tomRat = response_dict['tomatoRating']
                        tomatoRating.append(tomRat)
                        year = response_dict['Year']
                        Year.append(year)
                        gen = response_dict['Genre']
                        Genre.append(gen)
                        tomUM = response_dict['tomatoUserMeter']
                        tomatoUserMeter.append(tomUM)
                        dvd = response_dict['DVD']
                        DVD.append(dvd)
                        imdbRat = response_dict['imdbRating']
                        imdbRating.append(imdbRat)
                        imid = response_dict['imdbID']
                        imdbID.append(imid)
                        web = response_dict['Website']
                        Website.append(web)
                    else:   #otherwise append Null value to all of the columns, to maintain equal lists
                        Plot.append(None)
                        Rated.append(None)
                        Language.append(None)
                        Title.append(None)
                        Country.append(None)
                        tomatoMeter.append(None)
                        Writer.append(None)
                        tomatoURL.append(None)
                        BoxOffice.append(None)
                        tomatoUserRating.append(None)
                        Production.append(None)
                        Actors.append(None)
                        tomatoFresh.append(None)
                        Runtime.append(None)
                        Type.append(None)
                        imdbVotes.append(None)
                        Website.append(None)
                        tomatoConsensus.append(None)
                        tomatoReviews.append(None)
                        tomatoImage.append(None)
                        Poster.append(None)
                        tomatoRotten.append(None)
                        Metascore.append(None)
                        Response.append(None)
                        Director.append(None)
                        Released.append(None)
                        tomatoUserReviews.append(None)
                        Awards.append(None)
                        tomatoRating.append(None)
                        Year.append(None)
                        Genre.append(None)
                        tomatoUserMeter.append(None)
                        DVD.append(None)
                        imdbRating.append(None)
                        imdbID.append(None)
                except:
                    Plot.append(None)
                    Rated.append(None)
                    Language.append(None)
                    Title.append(None)
                    Country.append(None)
                    tomatoMeter.append(None)
                    Writer.append(None)
                    tomatoURL.append(None)
                    BoxOffice.append(None)
                    tomatoUserRating.append(None)
                    Production.append(None)
                    Actors.append(None)
                    tomatoFresh.append(None)
                    Runtime.append(None)
                    Type.append(None)
                    imdbVotes.append(None)
                    Website.append(None)
                    tomatoConsensus.append(None)
                    tomatoReviews.append(None)
                    tomatoImage.append(None)
                    Poster.append(None)
                    tomatoRotten.append(None)
                    Metascore.append(None)
                    Response.append(None)
                    Director.append(None)
                    Released.append(None)
                    tomatoUserReviews.append(None)
                    Awards.append(None)
                    tomatoRating.append(None)
                    Year.append(None)
                    Genre.append(None)
                    tomatoUserMeter.append(None)
                    DVD.append(None)
                    imdbRating.append(None)
                    imdbID.append(None)
            else:
                rated = response_dict['Rated']
                Rated.append(rated)
                plot = response_dict['Plot']
                Plot.append(plot)
                language = response_dict['Language']
                Language.append(language)
                title = response_dict['Title']
                Title.append(title)
                country = response_dict['Country']
                Country.append(country)
                tm = response_dict['tomatoMeter']
                tomatoMeter.append(tm)
                wr = response_dict['Writer']
                Writer.append(wr)
                turl = response_dict['tomatoURL']
                tomatoURL.append(turl)
                BO = response_dict['BoxOffice']
                BoxOffice.append(BO)
                tUR = response_dict['tomatoUserRating']
                tomatoUserRating.append(tUR)
                pro = response_dict['Production']
                Production.append(pro)
                act = response_dict['Actors']
                Actors.append(act)
                tf = response_dict['tomatoFresh']
                tomatoFresh.append(tf)
                rt = response_dict['Runtime']
                Runtime.append(rt)
                ty = response_dict['Type']
                Type.append(ty)
                imdbV = response_dict['imdbVotes']
                imdbVotes.append(imdbV)
                tomC = response_dict['tomatoConsensus']
                tomatoConsensus.append(tomC)
                tomR = response_dict['tomatoReviews']
                tomatoReviews.append(tomR)
                tomI = response_dict['tomatoImage']
                tomatoImage.append(tomI)
                post = response_dict['Poster']
                Poster.append(post)
                tomRot = response_dict['tomatoRotten']
                tomatoRotten.append(tomRot)
                meta = response_dict['Metascore']
                Metascore.append(meta)
                resp = response_dict['Response']
                Response.append(resp)
                dire = response_dict['Director']
                Director.append(dire)
                rele = response_dict['Released']
                Released.append(rele)
                tomURev = response_dict['tomatoUserReviews']
                tomatoUserReviews.append(tomURev)
                aw = response_dict['Awards']
                Awards.append(aw)
                tomRat = response_dict['tomatoRating']
                tomatoRating.append(tomRat)
                year = response_dict['Year']
                Year.append(year)
                gen = response_dict['Genre']
                Genre.append(gen)
                tomUM = response_dict['tomatoUserMeter']
                tomatoUserMeter.append(tomUM)
                dvd = response_dict['DVD']
                DVD.append(dvd)
                imdbRat = response_dict['imdbRating']
                imdbRating.append(imdbRat)
                imid = response_dict['imdbID']
                imdbID.append(imid)
                web = response_dict['Website']
                Website.append(web)
        except:
            x = row['api']
            print x + '4' 
            Plot.append(None)
            Rated.append(None)
            Language.append(None)
            Title.append(None)
            Country.append(None)
            tomatoMeter.append(None)
            Writer.append(None)
            tomatoURL.append(None)
            BoxOffice.append(None)
            tomatoUserRating.append(None)
            Production.append(None)
            Actors.append(None)
            tomatoFresh.append(None)
            Runtime.append(None)
            Type.append(None)
            imdbVotes.append(None)
            Website.append(None)
            tomatoConsensus.append(None)
            tomatoReviews.append(None)
            tomatoImage.append(None)
            Poster.append(None)
            tomatoRotten.append(None)
            Metascore.append(None)
            Response.append(None)
            Director.append(None)
            Released.append(None)
            tomatoUserReviews.append(None)
            Awards.append(None)
            tomatoRating.append(None)
            Year.append(None)
            Genre.append(None)
            tomatoUserMeter.append(None)
            DVD.append(None)
            imdbRating.append(None)
            imdbID.append(None)
    data['Plot'] = Plot
    data['Rated'] = Rated
    data['Language'] = Language
    data['Title'] = Title
    data['Country'] = Country
    data['tomatoMeter'] = tomatoMeter
    data['Writer'] = Writer
    data['tomatoURL'] = tomatoURL
    data['BoxOffice'] = BoxOffice
    data['tomatoUserRating'] = tomatoUserRating
    data['Production'] = Production
    data['Actors'] = Actors
    data['tomatoFresh'] = tomatoFresh
    data['Runtime'] = Runtime
    data['Type'] = Type
    data['imdbVotes'] = imdbVotes
    data['Website'] = Website
    data['tomatoConsensus'] = tomatoConsensus
    data['tomatoReviews'] = tomatoReviews
    data['tomatoImage'] = tomatoImage
    data['Poster'] = Poster
    data['tomatoRotten'] = tomatoRotten
    data['Metascore'] = Metascore
    data['Response'] = Response
    data['Director'] = Director
    data['Released'] = Released
    data['tomatoUserReviews'] = tomatoUserReviews
    data['Awards'] = Awards
    data['tomatoRating '] = tomatoRating 
    data['Year'] = Year
    data['Genre'] = Genre
    data['tomatoUserMeter'] = tomatoUserMeter
    data['DVD'] = DVD
    data['imdbRating'] = imdbRating
    data['imdbID'] = imdbID
    return data


In [16]:


#with open('finalMovie1_df.pkl', 'w') as picklefile:
    #pickle.dump(finalMovie1_df, picklefile)

In [17]:
#finalMovie2_df = pop_df(Complete_df[10000:20000])
#finalMovie2_df

#with open('finalMovie2_df.pkl', 'w') as picklefile:
    #pickle.dump(finalMovie2_df, picklefile)

In [18]:
#finalMovie3_df = pop_df(Complete_df[20000:25000])
#finalMovie3_df

#with open('finalMovie3_df.pkl', 'w') as picklefile:
    #pickle.dump(finalMovie3_df, picklefile)

In [19]:
#finalMovie4_df = pop_df(Complete_df[25000:30000])
#finalMovie4_df

#with open('finalMovie4_df.pkl', 'w') as picklefile:
    #pickle.dump(finalMovie4_df, picklefile)

In [20]:
#finalMovie5_df = pop_df(Complete_df[30000:35000])
#finalMovie5_df

#with open('finalMovie5_df.pkl', 'w') as picklefile:
    #pickle.dump(finalMovie5_df, picklefile)

In [21]:
#finalMovie6_df = pop_df(Complete_df[35000:39900])
#finalMovie6_df

#with open('finalMovie6_df.pkl', 'w') as picklefile:
    #pickle.dump(finalMovie6_df, picklefile)

In [23]:
#read all of the pickled dataframes

with open('finalMovie1_df.pkl', 'r') as picklefile:
    finalMovie1_df = pickle.load(picklefile)
with open('finalMovie2_df.pkl', 'r') as picklefile:
    finalMovie2_df = pickle.load(picklefile)
with open('finalMovie3_df.pkl', 'r') as picklefile:
    finalMovie3_df = pickle.load(picklefile)
with open('finalMovie4_df.pkl', 'r') as picklefile:
    finalMovie4_df = pickle.load(picklefile)
with open('finalMovie5_df.pkl', 'r') as picklefile:
    finalMovie5_df = pickle.load(picklefile)
with open('finalMovie6_df.pkl', 'r') as picklefile:
    finalMovie6_df = pickle.load(picklefile)

#list of populated dataframes
frames = [finalMovie2_df, finalMovie3_df, finalMovie4_df, finalMovie5_df, finalMovie6_df]

#append list of dataframes to the first one
FinalMovie_df = finalMovie1_df.append(frames)

#delete movies that were not found in IMDB, revisit later to have better retention
FinalMovie_df = FinalMovie_df[FinalMovie_df.imdbID.notnull()]

#drop unnecessary columns
FinalMovie_df.drop('title', axis=1, inplace= True)
FinalMovie_df.drop('genres', axis=1, inplace=True)
FinalMovie_df.drop('year', axis=1, inplace=True)
FinalMovie_df.drop('api-year', axis=1, inplace=True)
FinalMovie_df.drop('api', axis=1, inplace=True)

In [24]:
#create functions that create new column for each service and categorize each movie
#1 or 0 if movie found on that service
def netflixcol(x):
    if x == 'netflix':
        return 1
    else:
        return 0
def hulucol(x):
    if x == 'hulu':
        return 1
    else:
        return 0
def hbocol(x):
    if x == 'hbo':
        return 1
    else:
        return 0
def showcol(x):
    if x == 'showtime':
        return 1
    else:
        return 0
def amazoncol(x):
    if x == 'amazon':
        return 1
    else:
        return 0
    
#apply functions to the 'service' column and then delete it
FinalMovie_df['netflix'] = FinalMovie_df['service'].apply(netflixcol)
FinalMovie_df['hulu'] = FinalMovie_df['service'].apply(hulucol)
FinalMovie_df['hbo'] = FinalMovie_df['service'].apply(hbocol)
FinalMovie_df['showtime'] = FinalMovie_df['service'].apply(showcol)
FinalMovie_df['amazon'] = FinalMovie_df['service'].apply(amazoncol)

FinalMovie_df.drop('service', axis=1, inplace=True)

In [25]:
#now check for duplicate movies across different services and if duplicate merge the service
#columns

servicegrid_df = FinalMovie_df.groupby('Title')['netflix', 'hulu', 'hbo', 'showtime', 'amazon'].sum()
servicegrid_df = servicegrid_df.reset_index()

FinalMovie_df = FinalMovie_df.drop_duplicates('Title')

FinalMovie_df.drop('netflix', axis=1, inplace=True)
FinalMovie_df.drop('hulu', axis=1, inplace=True)
FinalMovie_df.drop('hbo', axis=1, inplace=True)
FinalMovie_df.drop('showtime', axis=1, inplace=True)
FinalMovie_df.drop('amazon', axis=1, inplace=True)

In [26]:
#final movie dataframe with duplicates removed
FinalMovie_df = pd.merge(FinalMovie_df, servicegrid_df)

with open('FinalMovie_df.pkl', 'w') as picklefile:
    pickle.dump(FinalMovie_df, picklefile)

In [22]:
#option to export to SQL, needed to be normalized and null values populated

#engine = create_engine('postgresql://KMS@localhost:5432/Capstone')
#*df*.to_sql('*df*(name of table)', engine)

# And you are done! You can now go to psql and explore your database and tables there. 
#If you want to import the data from the postgres database to python/pandas, see code below:

#%load_ext sql
#%%sql postgresql://KMS@localhost:5432/Capstone
        
#SELECT * FROM *df* LIMIT 5;
